In [0]:
%python
dbutils.widgets.text("catalog", "mpelletier")
dbutils.widgets.text("database", "dbdemos")
dbutils.widgets.text("warehouse_id", "1234")
dbutils.widgets.text("client_id", "1234")

In [0]:
%python
## CHANGE THESE VARIABLES AS NEEDED

catalog = dbutils.widgets.get("catalog")
database = dbutils.widgets.get("database")
warehouse_id = dbutils.widgets.get("warehouse_id")
client_id = dbutils.widgets.get("client_id")

print(f"{catalog}.{database}")

## Create and apply tags (Optional)

If you have enabled data classification, you can skip this step and go to **Create policies (ABAC)**

In [0]:
SET TAG ON COLUMN ${catalog}.${database}.students.name `class.name`;
SET TAG ON COLUMN ${catalog}.${database}.students.username `class.name`;
SET TAG ON COLUMN ${catalog}.${database}.students.address `class.location`;
SET TAG ON COLUMN ${catalog}.${database}.students.phone_number `class.phone_number`;
SET TAG ON COLUMN ${catalog}.${database}.students.email `class.email_address`;

## Explore existing tags

Use the these tables found under system.information_schema
- catalog_tags
- table_tags
- column_tags

In [0]:
SELECT DISTINCT(tag_name) FROM system.information_schema.table_tags
ORDER BY tag_name
LIMIT 10

### Assign multiple tags

`ALTER TABLE catalog.schema.table SET TAGS ('key1' = 'value1', 'key2' = 'value2');
`

## Create masking functions

In [0]:
-- Masks any input by returning a fully masked value
CREATE FUNCTION ${catalog}.${database}.mask_phone(value STRING)
RETURN '***-**-****' ;


In [0]:
-- hide rows from pubsec employees..
CREATE OR REPLACE FUNCTION ${catalog}.${database}.pubsec_email(email STRING)
RETURNS BOOLEAN
RETURN NOT LOWER(email) like '%.gov'

In [0]:
SELECT ${catalog}.${database}.pubsec_email('roger@pubsec.gov')

In [0]:
SELECT ${catalog}.${database}.mask_phone('123-45-6789')

## Create policies

### Create a SP for testing ABAC rules

In [0]:
%python
import time

from databricks.sdk import WorkspaceClient, AccountClient
from databricks.sdk.service import iam

w = WorkspaceClient()
a = AccountClient()

spn = w.service_principals.create(
    display_name=f"sp-abac-demo-{catalog}",
    groups=[],
)

spn

In [0]:
%python
spn_id = spn.application_id
spn_id

### Copy/paste SP id in widget

In [0]:
SELECT :spn_id

In [0]:
GRANT SELECT ON SCHEMA ${catalog}.${database} TO `${spn_id}`;
GRANT USAGE ON SCHEMA ${catalog}.${database} TO `${spn_id}`;

In [0]:
CREATE POLICY hide_pii_rows
ON TABLE ${catalog}.${database}.students
COMMENT 'Hide rows from pubsec employees'
ROW FILTER ${catalog}.${database}.pubsec_email
TO `${spn_id}`
FOR TABLES
MATCH COLUMNS
  hasTag('class.email_address') AS email_address
USING COLUMNS (email_address);


In [0]:
CREATE POLICY mask_information
ON TABLE ${catalog}.${database}.students
COMMENT 'Mask phone numbers'
COLUMN MASK ${catalog}.${database}.mask_phone
TO `${spn_id}`
FOR TABLES
MATCH COLUMNS
  hasTag('class.phone_number') as phone_number
ON COLUMN phone_number;



In [0]:
SHOW POLICIES ON SCHEMA ${catalog}.${database};

### Select data using current user
No column or row filters will be applied

In [0]:
SELECT * FROM ${catalog}.${database}.students

## Test

- with Service Principal
- with JDBC


### Create client secret for service principal

You will need to create client secret for the service principal
- Settings -> Identity and access -> Service principals -> Secrets
- Copy/paste the client id in the widget
- Copy/paste the client secret when prompted for the value

<img src="../data/sp_secrets.png" width="800" />

In [0]:
%python
client_id = dbutils.widgets.get("client_id")
client_id

In [0]:
%python
import getpass

client_secret = getpass.getpass("Enter service principal oauth secret: ")
print("Secret received.")


In [0]:
%python
import pandas as pd
from databricks.sdk import WorkspaceClient

# Replace these with your actual service principal details
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

# Initialize the WorkspaceClient with service principal authentication
w = WorkspaceClient(
    host=workspace_url,
    client_id=client_id,
    client_secret=client_secret
)

# Define the query to execute
query = f"SELECT * FROM {catalog}.{database}.students"

# Execute the query using execute_statement
# Warehouse ID
statement_response = w.statement_execution.execute_statement(query, warehouse_id)
#print(statement_response)

if statement_response.result:
    # Extract the data array from the statement response
    data_array = statement_response.result.data_array or []

    if data_array:
        # Convert the data array to a pandas DataFrame
        df = pd.DataFrame(data_array)

        # Display the DataFrame
        display(df)
    else:
        print("Query returned an empty dataset.")

## Clean-up resources

In [0]:
%python
user_input = input("Do you want to delete resources created in this lab (yes/no)?: ")
if user_input == 'no':
  raise Exception("OK")

In [0]:
%python
w.service_principals.delete(id=spn.id)

In [0]:
--DELETE FUNCTION
DROP FUNCTION ${catalog}.${database}.mask_phone;
DROP FUNCTION ${catalog}.${database}.pubsec_email;
DROP POLICY hide_pii_rows ON ${catalog}.${database}.students;
--UNSET TAG
DROP TABLE ${catalog}.${database}.students;
DROP SCHEMA ${catalog}.${database};
